In [1]:
import cv2
import numpy as np
import os
from scipy.misc import imread, imsave
import copy
import sys
os.chdir("../")
sys.path.append('/home/donghohan/compvision/new/22A_CV_Project_Team4/')
# sys.path.append(os.path.dirname(__file__) + "/../")
from VanishingPointUtils import *
from TeamClassificationUtils import *
from CoreOffsideUtils import *
from PoseEstimationUtils import *
import demo.demo_multiperson as PoseGetter
%matplotlib inline
# sys.path

/home/donghohan/.conda/envs/offside-detection/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/donghohan/.conda/envs/offside-detection/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/donghohan/.conda/envs/offside-detection/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/donghoh

['outputs_video', 'VanishingPointUtils.py', 'models', 'test', 'data', 'Pose_Utils', 'dataset', 'environment.yaml', 'backup', 'vis_dataset.py', '__pycache__', 'main.py', 'lib', 'CoreOffsideUtils.py', 'images', 'green.jpg', '.git', '.gitignore', 'main_video.py', 'outputs', 'parameters.json', 'video_track', 'LICENSE', 'eval_pck.py', 'demo', 'image_extraction', 'matlab', 'README.md', 'PoseEstimationUtils.py', 'nnet', 'compile.sh', 'TeamClassificationUtils.py', 'environment.yml', 'multiperson', 'util', 'PlayerSegmentationUtils.py']
Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Use standard file APIs to check for files with this prefix.


Instructions for updating:
Use standard file APIs to check for files with this prefix.


INFO:tensorflow:Restoring parameters from models/coco/coco-resnet-101


INFO:tensorflow:Restoring parameters from models/coco/coco-resnet-101


In [2]:
#Direction of goal
goalDirection = 'right'

keeper = [4.4931905696916325e-06, 4.450801979411523e-06, 5.510516736414265e-07, 0.00021567314734519837, 0.002188183807439825, 0.0015186984125557716, 0.7527352297592997, 1.0, 0.20787746170678337]
referee = [8.72783130847647e-06, 1.5868784197229944e-07, 0.0, 0.0010298840944002235, 0.0002880184331797235, 0.002688172043010753, 0.3064516129032258, 0.05913978494623656, 1.0]


In [3]:
base_path = '/home/donghohan/compvision/Computer-Vision-based-Offside-Detection-in-Soccer/dataset/Offside_Images/'
tempFileNames = os.listdir(base_path)
fileNames = []
for fileName in tempFileNames:
    fileNames.append(base_path+str(fileName))

In [4]:
def det(a, b):
    return a[0] * b[1] - a[1] * b[0]

def line_intersection(line1, line2):
    x_diff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
    y_diff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1])

    div = det(x_diff, y_diff)
    if div == 0:
        return None

    d = (det(*line1), det(*line2))
    x = det(d, x_diff) / div
    y = det(d, y_diff) / div

    return [x, y]

# def find_intersections(lines, height, width):
#     # print("This image's size:", height, width)
#     intersections = []
#     for i, line_1 in enumerate(lines):
#         for line_2 in lines[i + 1:]:
#             if not line_1 == line_2:
#                 intersection = line_intersection(line_1, line_2)
#                 if intersection:
#                     if intersection[0]<0 or intersection[0]>width or intersection[1]<0 or intersection[1]>height:
#                         intersections.append(intersection)
#                         # print("Intersection Point:"+str(intersection[0])+", "+str(intersection[1]))

#     return intersections

def find_intersections(lines):
    # print("This image's size:", height, width)
    intersections = []
    for i, line_1 in enumerate(lines):
        for line_2 in lines[i + 1:]:
            if not line_1 == line_2:
                intersection = line_intersection(line_1, line_2)
                if intersection:
                    # if intersection[0]<0 or intersection[0]>width or intersection[1]<0 or intersection[1]>height:
                    intersections.append(intersection)
                        # print("Intersection Point:"+str(intersection[0])+", "+str(intersection[1]))

    return intersections

In [5]:
def get_vertical_lines(image , side):
    img = image
    selectedLines = []
    selectedLinesParams = []
    linesFound = False
    BlueRedMask = 100
    
    
    
    
    while linesFound == False:
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv, (35, BlueRedMask, BlueRedMask), (70, 255,255))
        imask = mask>0
        green = np.zeros_like(img, np.uint8)
        green[imask] = img[imask]
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        v = np.median(green)
        sigma = 0.33
        lower = int(max(0, (1.0 - sigma) * v))
        upper = int(min(255, (1.0 + sigma) * v))
        cv2.imwrite('green.jpg', green)
        edges = cv2.Canny(green,150,250,apertureSize = 3) 
        minLineLength = 1
        maxLineGap = 1250
        lines = cv2.HoughLines(edges,1,np.pi/180, 200)
        if lines.any():
            if len(lines) > 2:  
                linesFound = True  
            else: 
                BlueRedMask -= 10

    
    
    linesFound = False
    
    if side == 'left':
    	angleMaxLimit = 20
    	angleMinLimit = 70
    else:
    	angleMaxLimit = 150
    	angleMinLimit = 105
    
    rLimit = 300
    while linesFound == False:
        for line in lines: 
            for r,theta in line:
                isLineValid = True
                a = np.cos(theta) 
                b = np.sin(theta)
                
                if (theta * 180 * 7 / 22) > angleMinLimit and (theta * 180 * 7 / 22) < angleMaxLimit:
                    #print(theta * 180 * 7 / 22 , r)
                    x0 = a*r 
                    y0 = b*r 
                    x1 = int(x0 + 1000*(-b)) 
                    y1 = int(y0 + 1000*(a)) 
                    x2 = int(x0 - 1000*(-b)) 
                    y2 = int(y0 - 1000*(a))
                    
                    #cv2.line(image,(x1,y1), (x2,y2), (0,0,255),1)
                    
                    
                    if len(selectedLines) > 0: 
                        for lineParams in selectedLinesParams:
                            #print(abs(lineParams[0] - r))
                            if abs(lineParams[0] - r) < rLimit:
                                isLineValid = False
                        for selectedLine in selectedLines:
                            if not line_intersection(selectedLine, [[x1,y1],[x2,y2]]):
                                isLineValid = False
                        if [[x1,y1],[x2,y2]] in selectedLines or [[x2,y2],[x1,y1]] in selectedLines:
                            isLineValid = False
                    if isLineValid:
                        #print("P")
                        selectedLines.append([[x1,y1],[x2,y2]])
                        print(x1,y1,x2,y2)
                        selectedLinesParams.append([r, theta])
                        cv2.line(image,(x1,y1), (x2,y2), (0,0,255),1)
                        cv2.putText(image, str((theta * 180 * 7 / 22)) ,(int((x2))  ,  int((y2))) , cv2.FONT_HERSHEY_SIMPLEX, 1, (200,255,155), 2, cv2.LINE_AA)
                        cv2.putText(image, str((theta * 180 * 7 / 22)) ,(int((x1))  ,  int((y1))) , cv2.FONT_HERSHEY_SIMPLEX, 1, (200,255,155), 2, cv2.LINE_AA)
        if len(selectedLines) < 2:
            if rLimit >= 75:
                rLimit -= 10
            else:
                angleMinLimit -= 1
                angleMaxLimit += 1
                rlimit = 100
        else:
            #for par in selectedLinesParams:
            #	print( "Angle", (par[1]*180*7)/22)
            linesFound = True
        
    return selectedLines

def get_horizontal_lines(image):
    img = image
    selectedLines = []
    selectedLinesParams = []
    linesFound = False
    BlueRedMask = 100
    
    while linesFound == False:
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv, (36, BlueRedMask, BlueRedMask), (100, 255,255))
        imask = mask>0
        green = np.zeros_like(img, np.uint8)
        green[imask] = img[imask]
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray,150,250,apertureSize = 3) 
        minLineLength = 1
        maxLineGap = 1250
        lines = cv2.HoughLines(edges,1,np.pi/180, 200)
        if lines.any():
            if len(lines) > 2:  
                linesFound = True  
            else: 
                BlueRedMask -= 10

    linesFound = False
    angleMaxLimit = 120
    angleMinLimit = 0
    # angleCosLimit = 0.5
    rLimit = 200
    while linesFound == False:
        for line in lines: 
            for r,theta in line:
                isLineValid = True
                a = np.cos(theta) 
                b = np.sin(theta)
                if (theta * 180 * 7 / 22) > angleMinLimit and (theta * 180 * 7 / 22) < angleMaxLimit:
                    x0 = a*r 
                    y0 = b*r 
                    x1 = int(x0 + 1000*(-b)) 
                    y1 = int(y0 + 1000*(a)) 
                    x2 = int(x0 - 1000*(-b)) 
                    y2 = int(y0 - 1000*(a))
                    if len(selectedLines) > 0: 
                        for lineParams in selectedLinesParams:
                            if abs(lineParams[0] - r) < rLimit:
                                isLineValid = False
                        for selectedLine in selectedLines:
                            if not line_intersection(selectedLine, [[x1,y1],[x2,y2]]):
                                isLineValid = False
                        if [[x1,y1],[x2,y2]] in selectedLines or [[x2,y2],[x1,y1]] in selectedLines:
                            isLineValid = False
                    if isLineValid:
                        selectedLines.append([[x1,y1],[x2,y2]])
                        selectedLinesParams.append([r, theta])
                        # cv2.line(image,(x1,y1), (x2,y2), (0,0,255),1)
        if len(selectedLines) < 2:
            if rLimit >= 75:
                rLimit -= 10
            else:
                angleMinLimit -= 1
                angleMaxLimit += 1
            # angleCosLimit -= 0.05 
        else:
            linesFound = True
            
    return selectedLines



def get_vertical_vanishing_point(img , side):
    
    selectedLines = get_vertical_lines(img , side)
    intersectionPoints = find_intersections(selectedLines)
    vanishingPointX = 0.0
    vanishingPointY = 0.0
    for point in intersectionPoints:
        vanishingPointX += point[0]
        vanishingPointY += point[1]
    # print(selectedLines)

    return (vanishingPointX/len(intersectionPoints), vanishingPointY/len(intersectionPoints))

def get_horizontal_vanishing_point(img):
    
    selectedLines = get_horizontal_lines(img)
    intersectionPoints = find_intersections(selectedLines)
    vanishingPointX = 0.0
    vanishingPointY = 0.0
    for point in intersectionPoints:
        vanishingPointX += point[0]
        vanishingPointY += point[1]

    return (vanishingPointX/len(intersectionPoints), vanishingPointY/len(intersectionPoints))

In [7]:
teamminus = []
teamsecond = []
for file_itr in range(len(fileNames)):
	# if(file_itr == 1):
	# 	break
	# if(fileNames[file_itr]!='/home/donghohan/compvision/Computer-Vision-based-Offside-Detection-in-Soccer/dataset/Offside_Images/331.jpg'):
	# 	continue
	# 	if(cnt==1):		
	# 		continue
	# if(cnt==0):
	# 	cnt=1
 
	# if(fileNames[file_itr]!='/home/donghohan/compvision/Computer-Vision-based-Offside-Detection-in-Soccer/dataset/Offside_Images/6.jpg'):
	# 	continue
	print('\n\n', fileNames[file_itr])
	if not tempFileNames[file_itr].endswith('.jpg'):
		continue
	# calculate vanishing points
	imageForVanishingPoints = cv2.imread(fileNames[file_itr])
	vertical_vanishing_point = get_vertical_vanishing_point(imageForVanishingPoints, goalDirection)
	horizontal_vanishing_point = get_horizontal_vanishing_point(imageForVanishingPoints)
	# cv2.imwrite(vanishing_point_viz_base_path+tempFileNames[file_itr], imageForVanishingPoints)
	print('Finished Vanishing Point calculation')
	# get pose estimaitons and team classifications
	imageForPoseEstimation = cv2.imread(fileNames[file_itr])
	imageForPoseEstimation_2 = imread(fileNames[file_itr], mode='RGB')
	# contour check
	# _, contour_image = PoseGetter.get_contours(imageForPoseE1stimation)
	# cv2.imwrite(contour_path+tempFileNames[file_itr], contour_image)
 	
	pose_estimations1, isKeeperFound, isRefFound, temp_image = PoseGetter.return_pose(imageForPoseEstimation_2, imageForPoseEstimation, keeper, referee)
	# cv2.imwrite(base_path+'sub/'+tempFileNames[file_itr], temp_image)
 
	for pose in pose_estimations1:
		if pose[1]==-1:
			print("teamId:-1 detected in ", tempFileNames[file_itr])
			teamminus.append(tempFileNames[file_itr])
		elif pose[1] == 2:
			print("teamId:2 detected in ", tempFileNames[file_itr])
			teamsecond.append(tempFileNames[file_itr])
	continue

 
 
	pose_estimations2 = sorted(pose_estimations1, key=lambda x : x[-1][0])
	pose_estimations3 = copy.deepcopy(pose_estimations2)
	pose_estimations3 = update_pose_left_most_point(vertical_vanishing_point, horizontal_vanishing_point, pose_estimations3, imageForPoseEstimation, goalDirection)
	print('Finished Pose Estimation & Team Classifiaciton')
 
 
 
	# pose_estimations structure -> [id, teamId, keyPoints, leftmostPoint]
	pose_estimations4 = copy.deepcopy(pose_estimations3)
	pose_estimations4 = get_leftmost_point_angles(vertical_vanishing_point, pose_estimations4, imageForPoseEstimation, goalDirection)
	print('Finished updating leftmost point using angle')
 
 
 
	# pose_estimations structure -> [id, teamId, keyPoints, leftmostPoint, angleAtVanishingPoint]
	pose_estimations5 = sorted(pose_estimations4, key=lambda x : x[-1])
 
 
	# font = cv2.FONT_HERSHEY_SIMPLEX
	# for pose in pose_estimations5:
	# 	cv2.putText(imageForPoseEstimation, str(str(pose[0])), (int(pose[-2][-1]), int(pose[-2][0])), font, 1, (200,255,155), 2, cv2.LINE_AA)
	# 	cv2.line(imageForPoseEstimation, (int(vertical_vanishing_point[0]) , int(vertical_vanishing_point[1])) , (int(pose[-2][-1]), int(pose[-2][0])), (0,255,0) , 2 )
	# # cv2.imwrite(pose_estimation_viz_base_path+tempFileNames[file_itr], imageForPoseEstimation)
	# # visualize teams
	# imageForTeams = cv2.imread(fileNames[file_itr])
	# for pose in pose_estimations5:
	# 	font = cv2.FONT_HERSHEY_SIMPLEX
	# 	cv2.putText(imageForTeams, str(pose[1]), (int(pose[-2][-1]), int(pose[-2][0])), font, 1, (200,255,155), 2, cv2.LINE_AA)
	# # cv2.imwrite(team_classification_viz_base_path+tempFileNames[file_itr], imageForTeams)
 
 
 
 
 
 
 
	# get offside decisions
	pose_estimations6 = copy.deepcopy(pose_estimations5)
	pose_estimations6, last_defending_man = get_offside_decision(pose_estimations6, vertical_vanishing_point, 0, 1, isKeeperFound)
	# # pose_estimations structure -> [id, teamId, keyPoints, leftmostPoint, angleAtVanishingPoint, offsideDecision]
	# print('Starting Core Offside Algorithm')
	# imageForOffside = cv2.imread(fileNames[file_itr])
	# for pose in pose_estimations:
	# 	if pose[1] == 0:
	# 		if pose[-1] == 'off':
	# 			font = cv2.FONT_HERSHEY_SIMPLEX
	# 			cv2.putText(imageForOffside, 'off', (int(pose[-3][-1]), int(pose[-3][0]-10)), font, 1, (200,255,155), 2, cv2.LINE_AA)
	# 			cv2.line(imageForOffside , (int(vertical_vanishing_point[0]) , int(vertical_vanishing_point[1])) , (int(pose[-3][-1]), int(pose[-3][0])), (0,255,0) , 2 )
	# 		else:
	# 			font = cv2.FONT_HERSHEY_SIMPLEX
	# 			cv2.putText(imageForOffside, 'on', (int(pose[-3][-1]), int(pose[-3][0]-10)), font, 1, (200,255,155), 2, cv2.LINE_AA)
	# 	elif pose[1] == 1:
	# 		if pose[0] == last_defending_man:
	# 			cv2.putText(imageForOffside, 'last man', (int(pose[-3][-1]), int(pose[-3][0]-15)), font, 1, (200,255,155), 2, cv2.LINE_AA)
	# 			cv2.line(imageForOffside , (int(vertical_vanishing_point[0]) , int(vertical_vanishing_point[1])) , (int(pose[-3][-1]), int(pose[-3][0])), (0,255,0) , 2 )
	# 		else:
	# 			cv2.putText(imageForOffside, 'def', (int(pose[-3][-1]), int(pose[-3][0]-15)), font, 1, (200,255,155), 2, cv2.LINE_AA)
	# 			cv2.line(imageForOffside , (int(vertical_vanishing_point[0]) , int(vertical_vanishing_point[1])) , (int(pose[-3][-1]), int(pose[-3][0])), (0,255,0) , 2 )				
	# 	elif pose[1] == 2:
	# 		cv2.putText(imageForOffside, 'keep', (int(pose[-3][-1]), int(pose[-3][0]-10)), font, 1, (200,255,155), 2, cv2.LINE_AA)
	# 		cv2.line(imageForOffside , (int(vertical_vanishing_point[0]) , int(vertical_vanishing_point[1])) , (int(pose[-3][-1]), int(pose[-3][0])), (0,255,0) , 2 )
	# 	elif pose[1] == 3:
	# 		cv2.putText(imageForOffside, 'ref', (int(pose[-3][-1]), int(pose[-3][0]-10)), font, 1, (200,255,155), 2, cv2.LINE_AA)
	# 		cv2.line(imageForOffside , (int(vertical_vanishing_point[0]) , int(vertical_vanishing_point[1])) , (int(pose[-3][-1]), int(pose[-3][0])), (0,255,0) , 2 )
	# cv2.imwrite(offside_viz_base_path+tempFileNames[file_itr][:-4]+'_1.jpg', imageForOffside)
	
	# print(file_itr,fileNames[file_itr])



 /home/donghohan/compvision/Computer-Vision-based-Offside-Detection-in-Soccer/dataset/Offside_Images/340.jpg
-1048 665 946 804
-1025 459 971 564
Finished Vanishing Point calculation
normes dist [0.10952800716459592, 0.13280852837664398, 0.07250028260922858, 0.11812121873229373, 0.0790352006867348, 0.04878361147375781, 0.0638697612367609, 0.048605662700106404, 0.09988900897383174, 0.10428602301876252, 0.047371236047866964, 0.07520145897941669]
ref: [] keeper: -1 score 0.25


 /home/donghohan/compvision/Computer-Vision-based-Offside-Detection-in-Soccer/dataset/Offside_Images/470.jpg
-830 -584 981 260
-840 -671 1072 -86
Finished Vanishing Point calculation
normes dist [0.040036991774412306, 0.04892004531380976, 0.048860799406638, 0.060673074728080784, 0.06727939365150348, 0.052732399037202705, 0.044497008745268365, 0.0670653707985963, 0.05147484087329714, 0.06870133968479908, 0.09679911093628386, 0.04909038896017687, 0.0626682613438669, 0.06024762818837717, 0.05781438650932613, 0.065493

KeyboardInterrupt: 

In [ ]:
for i in range(4):
    print("This component is")
    print(pose_estimations1[0][i])

This component is
0
This component is
1
This component is
{'leftShoulder': {'x': 225.26937985420224, 'y': 1278.669126033783}, 'rightShoulder': {'x': 225.7564980983734, 'y': 1271.216631412506}, 'leftHip': {'x': 250.2277148962021, 'y': 1284.834002673626}, 'rightHip': {'x': 249.5440754890442, 'y': 1278.915053844452}, 'leftKnee': {'x': 272.86796140670776, 'y': 1279.787676334381}, 'rightKnee': {'x': 272.38460969924927, 'y': 1278.970989704132}, 'leftAnkle': {'x': 284.24081195890903, 'y': 1282.5642998218536}, 'rightAnkle': {'x': 283.72224152088165, 'y': 1281.6826508045197}}
This component is
[225.26937985420224, 1278.669126033783]
